In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
import os
import glob

In [1]:
measure = 'retrieved_ids/cosine'

In [7]:
path = '/media/sebastian/Seagate Backup Plus Drive/MMSR/task2/'
id_information = pd.read_csv(path + 'id_information_mmsr.tsv', sep='\t')

In [3]:
file_list = []
for file in os.listdir("./retrieved_ids/combinations"):
    if file.endswith(".pkl"):
        file_list.append(file)
    
print(file_list)

['mfcc_bow_cosine_retrieved_ids.pkl', 'resnet_cosine_retrieved_ids.pkl']


In [4]:
dict_dict = {}
for file in file_list:
    with open('./retrieved_ids/combinations/' + file, 'rb') as f:
        retrieved_dict = pickle.load(f)
        dict_dict[file] = retrieved_dict

Different weights for different featrues, adapt accordingly

In [17]:
weights = [0.603, 0.548, 0.606, 0.593, 0.501]

In [36]:
weights = [0.538, 0.543, 0.566, 0.578, 0.539]

In [127]:
weights = [0.603, 0.574, 0.606]

In [115]:
weights = [0.593, 0.564, 0.577]

In [92]:
weights = [0.603, 0.538, 0.548, 0.543, 0.606, 0.566, 0.578, 0.539, 0.593, 0.501]

In [128]:
def normal(x):
    return (x-0.494)/(0.606 - 0.494)

In [129]:
nor_weight = []
for x in weights:
    print(normal(x))
    nor_weight.append(normal(x))

0.9732142857142857
0.714285714285714
1.0


In [8]:
borda_dict = {}
for song in tqdm(id_information['id']):
    vote_dict = {}
    for idc, method in enumerate(file_list): #[1:]:
        for vote, idx in enumerate(dict_dict[method][song]):
            if idx in vote_dict:
                vote_dict[idx] += nor_weight[idc]*(100-vote)
            else:
                vote_dict[idx] = nor_weight[idc]*(100-vote)
    borda_dict[song] = vote_dict

100%|██████████████████████████████████| 68641/68641 [00:04<00:00, 16137.14it/s]


In [11]:
new_dict = {}
for song in tqdm(id_information['id']):
    borda_df = pd.DataFrame.from_dict(borda_dict[song], orient='index', columns=['counts'])
    borda_df = borda_df.sort_values('counts', ascending=False)
    new_list = list(borda_df.index.values)[:100]
    new_dict[song] = new_list

100%|███████████████████████████████████| 68641/68641 [00:33<00:00, 2044.78it/s]


In [12]:
with open('simple_borda_best.pkl', 'wb') as f:
    pickle.dump(new_dict, f)

In [139]:
simple_borda_df = pd.DataFrame.from_dict(new_dict, orient='index')

In [11]:
simple_borda_df.to_csv('simple_borda.tsv', sep='\t')

In [140]:
simple_borda_df.to_csv('weighted_borda_10.csv', sep=',')